# Design and Analyze a beam with holes

The following example demonstrates how you can add penetrations to a beam using various primitives and finally export it to an IFC file

In [3]:
import numpy as np
from ada import Assembly, Beam, Part, PrimBox, PrimCyl, PrimExtrude, Material, CarbonSteel
from ada.fem import Bc, FemSet, Load, Step
from ada.fem.io.mesh.recipes import create_beam_mesh
from ada.fem.utils import get_beam_end_nodes

**The basic beam setup**

The following code creates the following Assembly (and IFC) hierarchy

    MyAssembly (IfSite)
        MyPart (IfcBuildingStorey)
            MyBeam (IfcBeam)

In [4]:
bm = Beam("MyBeam", (0, 0, 0), (1.5, 0, 0), "IPE400", Material('S420', CarbonSteel("S420")))
a = Assembly("MyAssembly") / [Part("MyPart") / bm]
a

Assembly("MyAssembly": Beams: 1, Plates: 0, Pipes: 0, Shapes: 0, Elements: 0, Nodes: 0)

**Polygon Extrusions**

In [5]:
h = 0.2
r = 0.02

normal = [0, 1, 0]
xdir = [-1, 0, 0]


origin = np.array([0.2, -0.1, -0.1])
points = [(0, 0), (0.1, 0), (0.05, 0.1)]

poly1 = bm.add_penetration(PrimExtrude("Poly1", points, h, normal, origin, xdir))
bm

Beam(id: 0Ds0yek8eHwuMZyDM$DI$q, Name: MyBeam
N1: Node([0.0, 0.0, 0.0], 1), N2: Node([1.5, 0.0, 0.0], 2)
Section: Section(IPE400, IPE, h: 0.4, w_btn: 0.18, w_top: 0.18, t_fbtn: 0.0135, t_ftop: 0.0135, t_w: 0.0086)
Material: Material(Name: "S355" Material Model: "CarbonSteel(E:2.100E+11, sig_y:3.550E+08, rho:7.850E+03, plasticity_model:None))

In [6]:
origin += np.array([0.2, 0, 0])
points = [(0, 0, r), (0.1, 0, r), (0.05, 0.1, r)]

poly2 = bm.add_penetration(PrimExtrude("Poly2", points, h, normal, origin, xdir))
bm

Beam(id: 0Ds0yek8eHwuMZyDM$DI$q, Name: MyBeam
N1: Node([0.0, 0.0, 0.0], 1), N2: Node([1.5, 0.0, 0.0], 2)
Section: Section(IPE400, IPE, h: 0.4, w_btn: 0.18, w_top: 0.18, t_fbtn: 0.0135, t_ftop: 0.0135, t_w: 0.0086)
Material: Material(Name: "S355" Material Model: "CarbonSteel(E:2.100E+11, sig_y:3.550E+08, rho:7.850E+03, plasticity_model:None))

In [7]:
origin += np.array([0.2, 0, 0])
points = [(0, 0, r), (0.1, 0, r), (0.1, 0.2, r), (0.0, 0.2, r)]

poly3 = bm.add_penetration(PrimExtrude("Poly3", points, h, normal, origin, xdir))
bm

Beam(id: 0Ds0yek8eHwuMZyDM$DI$q, Name: MyBeam
N1: Node([0.0, 0.0, 0.0], 1), N2: Node([1.5, 0.0, 0.0], 2)
Section: Section(IPE400, IPE, h: 0.4, w_btn: 0.18, w_top: 0.18, t_fbtn: 0.0135, t_ftop: 0.0135, t_w: 0.0086)
Material: Material(Name: "S355" Material Model: "CarbonSteel(E:2.100E+11, sig_y:3.550E+08, rho:7.850E+03, plasticity_model:None))

In [8]:
# Cylinder Extrude
x = origin[0] + 0.2

cyl = bm.add_penetration(PrimCyl("cylinder", (x, -0.1, 0), (x, 0.1, 0), 0.1))
bm

Beam(id: 0Ds0yek8eHwuMZyDM$DI$q, Name: MyBeam
N1: Node([0.0, 0.0, 0.0], 1), N2: Node([1.5, 0.0, 0.0], 2)
Section: Section(IPE400, IPE, h: 0.4, w_btn: 0.18, w_top: 0.18, t_fbtn: 0.0135, t_ftop: 0.0135, t_w: 0.0086)
Material: Material(Name: "S355" Material Model: "CarbonSteel(E:2.100E+11, sig_y:3.550E+08, rho:7.850E+03, plasticity_model:None))

In [9]:
# Box Extrude
x += 0.2

box = bm.add_penetration(PrimBox("box", (x, -0.1, -0.1), (x + 0.2, 0.1, 0.1)))
bm

Beam(id: 0Ds0yek8eHwuMZyDM$DI$q, Name: MyBeam
N1: Node([0.0, 0.0, 0.0], 1), N2: Node([1.5, 0.0, 0.0], 2)
Section: Section(IPE400, IPE, h: 0.4, w_btn: 0.18, w_top: 0.18, t_fbtn: 0.0135, t_ftop: 0.0135, t_w: 0.0086)
Material: Material(Name: "S355" Material Model: "CarbonSteel(E:2.100E+11, sig_y:3.550E+08, rho:7.850E+03, plasticity_model:None))

In [10]:
# Export IFC to the Home folder
a.to_ifc('../output/MyBeamWithHoles.ifc')
a

ifc file created at "..\output\MyBeamWithHoles.ifc"


Assembly("MyAssembly": Beams: 1, Plates: 0, Pipes: 0, Shapes: 0, Elements: 0, Nodes: 0)

In [11]:
# Create a FEM analysis of the beam as a cantilever subjected to gravity loads
p = a.get_part("MyPart")
create_beam_mesh(bm, p.fem, "shell")

# Add a set containing ALL elements (necessary for Calculix loads).
fs = p.fem.add_set(FemSet("Eall", [el for el in p.fem.elements], "elset"))

step = a.fem.add_step(Step("gravity", "static", nl_geom=True, init_incr=100.0, total_time=100.0))
step.add_load(Load("grav", "gravity", -9.81 * 800, fem_set=fs))

fix_set = p.fem.add_set(FemSet("bc_nodes", get_beam_end_nodes(bm), "nset"))
a.fem.add_bc(Bc("Fixed", fix_set, [1, 2, 3]))

a

step file created at "C:\Users\ofskrand\ADA\temp\MyBeam_0O5EXOk8eHwv9oyDM$DI$q.stp"


Assembly("MyAssembly": Beams: 1, Plates: 0, Pipes: 0, Shapes: 0, Elements: 513, Nodes: 309)

In [12]:
res = a.to_fem("MyCantilever_code_aster", "code_aster", execute=True)

Exporting to "code_aster" using to_fem
creating: MyCantilever_code_aster


In [ ]:
res